# Demo Notebook how to run models on static mouse datasets

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [2]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True

name = "test"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [3]:
import torch
import numpy as np
import pickle 
import pandas as pd
from collections import OrderedDict, Iterable

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder

from nnsysident.tables.experiments import *
from nnsysident.datasets.mouse_loaders import static_shared_loaders
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.datasets.mouse_loaders import static_loader

<ipython-input-3-71e7f2af38a6>:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import OrderedDict, Iterable


Connecting konstantin@134.2.168.16:3306


# Get Dataloader

In [4]:
# change path here
paths = ['data/static22564-2-12-preproc0.zip',
                     'data/static22564-2-13-preproc0.zip',
                     'data/static22564-3-8-preproc0.zip',
                     'data/static22564-3-12-preproc0.zip']

#paths = ['data/static22564-2-12-preproc0.zip']
dataset_fn = 'nnsysident.datasets.mouse_loaders.static_shared_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=64,
    seed=1,
    #image_n=50,
    #image_base_seed=1,
    multi_match_n=972,
    multi_match_base_seed=1,
    exclude_multi_match_n = 3625,

)

dataloaders = builder.get_data(dataset_fn, dataset_config)

data/static22564-2-12-preproc0 exists already. Not unpacking data/static22564-2-12-preproc0.zip
data/static22564-2-13-preproc0 exists already. Not unpacking data/static22564-2-13-preproc0.zip
data/static22564-3-8-preproc0 exists already. Not unpacking data/static22564-3-8-preproc0.zip
data/static22564-3-12-preproc0 exists already. Not unpacking data/static22564-3-12-preproc0.zip
data/static22564-2-12-preproc0 exists already. Not unpacking data/static22564-2-12-preproc0.zip
data/static22564-2-13-preproc0 exists already. Not unpacking data/static22564-2-13-preproc0.zip
data/static22564-3-8-preproc0 exists already. Not unpacking data/static22564-3-8-preproc0.zip
data/static22564-3-12-preproc0 exists already. Not unpacking data/static22564-3-12-preproc0.zip


# Get Model

### The New gaussian readout: change gauss_type for the different modes

In [5]:
model_fn = 'nnsysident.models.models.se2d_fullgaussian2d'

model_config = {'share_features': True,
                'share_transform': True,
                 'init_mu_range': 0.55,
                 'init_sigma': 0.4,
                 'input_kern': 15,
                 'hidden_kern': 13,
                 'gamma_input': 1.0,
                 'gamma_readout': 0.333593, #2.117604964706911, #626.6499356203459, # 2.117604964706911, #626.6499356203459
                 'grid_mean_predictor': {'type': 'cortex',
                                          'input_dimensions': 2,
                                          'hidden_layers': 0,
                                          'hidden_features': 0,
                                          'final_tanh': False,
                                        }}


model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)
model_real = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)
model_start = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)

/src/nnsysident/nnsysident/models/models.py:141: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  all_multi_unit_ids = set(np.hstack(shared_match_ids.values()))


In [ ]:
#core_dict = OrderedDict([(k, v) for k, v in torch.load('dfc6a9fbcca790d7a6b59ff787d96356.pth.tar').items() if k[0:5] == 'core.'])
core_dict = torch.load('21ee7ec25ed7fc959838386c61f5c91a.pth.tar')

def get_grids(my_model):
    grids = {}
    for key, readout in my_model.readout.items():
        grid = readout.grid.squeeze().cpu().data.numpy()
        grids[key] = grid
    return grids

list_of_load = [] #'scales', '_features' ,'mu_transform.0.weight', 'mu_transform.0.bias'
list_of_detach = []



model_real.load_state_dict(core_dict, strict=False)
real_grids = get_grids(model_real)

remove=[]
keep=[]
for key in core_dict.keys():
    name = '.'.join(key.split('.')[2:])
    if key.split('.')[0] == 'readout': 
        if not np.isin(name, list_of_load):
            print('Not loading:    {}'.format(key))
            remove.append(key)
        else:
            keep.append(key)
for key in keep:
    print('Loading:  {}'.format(key))

for k in remove: del core_dict[k]
model_start.load_state_dict(core_dict, strict=False)
start_grids = get_grids(model_start)



model.load_state_dict(core_dict, strict=False)

for param in model.named_parameters():
    name = '.'.join(param[0].split('.')[2:])
    if param[0].split('.')[0] == 'readout':
        if np.isin(name, list_of_detach):
            print('detaching:    {}'.format(param[0]))
            param[1].requires_grad = False

# Get Trainer

In [6]:
trainer_fn = 'nnsysident.training.trainers.standard_trainer'
trainer_config = dict(track_training=True) #detach_core=True, 
trainer = builder.get_trainer(trainer_fn, trainer_config)

# Run Training

In [7]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1)

correlation 0.0019739168
poisson_loss 2381225.8


Epoch 1: 100%|██████████| 280/280 [00:26<00:00, 10.42it/s]


[001|00/05] ---> 0.11457011848688126
correlation 0.11457012
poisson_loss 1215937.2


Epoch 2: 100%|██████████| 280/280 [00:10<00:00, 27.09it/s]


[002|00/05] ---> 0.16468699276447296
correlation 0.164687
poisson_loss 1177759.2


...

[062|03/05] -/-> 0.3716176450252533
correlation 0.37161765
poisson_loss 988323.9


Epoch 63: 100%|██████████| 280/280 [00:10<00:00, 27.34it/s]


[063|04/05] -/-> 0.37281760573387146
correlation 0.3728176
poisson_loss 986451.75


Epoch 64: 100%|██████████| 280/280 [00:10<00:00, 27.99it/s]


[064|05/05] -/-> 0.3726043403148651
Restoring best model after lr decay! 0.372604 ---> 0.373510
Restoring best model! 0.373510 ---> 0.373510


Best old:                                  0.376 - 0.379  
Best new (no shared position):             0.377  
Best new (with shared position):           0.373510  

In [9]:
from torch.nn import Parameter

a = Parameter(torch.Tensor(1, 10, 1, 1))

In [13]:
a.data.normal_(1, 1.e-3)

tensor([[[[1.0010]],

         [[0.9991]],

         [[0.9998]],

         [[0.9999]],

         [[0.9995]],

         [[0.9991]],

         [[0.9998]],

         [[1.0010]],

         [[1.0010]],

         [[1.0003]]]])

In [ ]:
grids = get_grids(model)
    
from nnsysident.utility.measures import get_correlations

cors = get_correlations(model, dataloaders['test'], per_neuron=False, as_dict=True)
for key, value in cors.items():
    cors[key] = np.mean(value)

In [ ]:
title = 'feat|scale: random init + learned - position: random 0.33 ortho init|0 + learned + shared'

dictionary = dict(cors=cors,
                  grids = grids,
                  real_grids = real_grids,
                  start_grids = start_grids,)
#torch.save(dictionary, title)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 2, dpi=200, figsize=(6, 4.3))

for ind, (ax, (key, grid), (real_key, real_grid), (start_key, start_grid)) in enumerate(zip(axes.flat, grids.items(), real_grids.items(), start_grids.items())):
    assert key==real_key==start_key, 'keys do not match'
    ax.scatter(*grid.T, color='navy', s=.1, label='Learned position')
    ax.scatter(*real_grid.T, color='red', s=.1, label='"True" position')
    ax.scatter(*start_grid.T, color='black', s=.1, label='Init position', alpha=0.2)

    ax.set(xlim=(-1.1, 1.1), ylim=(-1.1, 1.1), label='Learned correlation')
    ax.text(-1,.8, round(cors[key], 3), fontsize=8.5, color='navy')
    sns.despine(top=True, ax=ax)
    
    ax.set(xticks=[-1, -.5, 0, .5, 1])
    ax.set_axisbelow(True)
    ax.grid(ls='--')
    if ind != 3:
        ax.set(xticklabels=[], yticklabels=[])
        
    ax.set_title(key, fontsize=10)
fig.suptitle(title.split('-')[0] + '\n' + title.split('-')[-1], y=1.05, fontsize=10.)

plt.tight_layout()
#plt.savefig(title + '.png', dpi=200, bbox_inches='tight')